# <center>Organizaci&oacute;n de Datos</center>
## <center>Trabajo Práctico 2: Criticas de cine</center>

#### Integrantes grupo 19: Sudanalytics 
* Adrian Leandro Re
* Alan Toulouse
* Luciano Lorenzo
* Nicolas Tonizzo

Importamos las librerias y modulos necesarios.

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import make_scorer, f1_score, confusion_matrix, precision_score, recall_score, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasClassifier

Cargamos nuestro set de entrenamiento

In [3]:
df = pd.read_csv("dataset/train.csv")
df.head()

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",positivo


## Feature Engineering

Encodeamos el target para que las review positivas esten representadas por un 1 y las negativas por 0.

In [4]:
df['sentimiento_es_positivo'] = np.where(df.sentimiento == 'positivo', 1, 0)

## Sets de test y train
Dividimos el dataframe en datos de test y train. Usaremos estos mismos sets para el entrenamiento de todos los modelos.

In [5]:
X = df.review_es
y = df.sentimiento_es_positivo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)

## Ensamblado de modelos

### Bayes Naive
Primero transformamos nuestros documentos en una matriz en la cual se almacena la cantidad de tokens ("palabras clave"). Luego convertimos esa misma matriz en una representación normalizada de término/frecuencia. 
Esto lo hacemos para reducir el impacto de los tokens que aparecen más veces.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

vectorizador = CountVectorizer()
X_train_counts = vectorizador.fit_transform(X_train)

escaladorTfidf = TfidfTransformer()
X_train_tfidf = escaladorTfidf.fit_transform(X_train_counts)

Ahora que tenemos la información en el formato correcto, creamos nuestro modelo de tipo Bayes Naive. En este caso usamos uno de tipo multinomial.

In [7]:
clasificadorBayesNaive = MultinomialNB()
clasificadorBayesNaive.fit(X_train_tfidf, y_train)

MultinomialNB()

Realizamos el mismo procedimiento para transformar los datos de validación y poder realizar nuestras predicciones.

In [8]:
X_test_counts = vectorizador.transform(X_test)
X_test_tfidf = escaladorTfidf.transform(X_test_counts)
y_pred = np.where(clasificadorBayesNaive.predict(X_test_tfidf) >= 0.5, 1, 0)

Construimos nuestra matriz de confusión y analizamos métricas

Buscamos ahora los mejores parámetros para nuestro modelo

In [9]:
from sklearn.pipeline import Pipeline

text_clf = Pipeline([
     ('vect', vectorizador),
     ('tfidf', escaladorTfidf),
     ('clf', clasificadorBayesNaive),
])


parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': (1e-1,1e-2, 1e-3),
}

In [10]:
gs_clf = RandomizedSearchCV(text_clf, parameters, cv=5, n_jobs=-1)

In [ ]:
nb_clf = gs_clf.fit(X_train, y_train)

In [ ]:
nb_clf.best_score_

In [ ]:
confusion = confusion_matrix(y_test, y_pred)
sns.heatmap(confusion, annot=True, fmt='.0f')
plt.ylabel('true')
plt.xlabel('predicted')
plt.title("Confusion matrix de validación")

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))

Creamos el submit para la competencia de Kaggle.

In [ ]:
df_test = pd.read_csv("dataset/test.csv")
assert(len(df_test)==8599)

X = df_test['review_es']
    
X_counts = vectorizador.transform(X)
X_tfidf = escaladorTfidf.transform(X_counts)
    
prediccion = np.where(model.predict(X_tfidf) >= 0.5, 'positivo', 'negativo')

assert(len(prediccion)==8599)

submit = pd.DataFrame(
    {"sentimiento": prediccion},
    index=df_test["ID"],
)
assert(len(submit)==8599)

submit.to_csv("dataset/submit.csv")

### Random Forest

### XGBoost

### Red neuronal con Keras y TensorFlow

### Ensamble de modelos